In [2]:
import torch

In [418]:
k = 30

In [577]:
mu = torch.randn(k,  requires_grad=True)
optimizer = torch.optim.SGD([mu], lr=1e-2)
for t in range(0,10000):
    x = torch.cat((torch.tensor([1.0]), torch.sigmoid(mu)))
    z = torch.cat((x[-1:], x[:-1])) 
    equi = x * z + (1-x) * (1-z)
    all_equi = torch.prod(equi)
    loss = -torch.log(all_equi)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
torch.round(x*100)

tensor([100.,  99.,  99.,  50.,   1.,   1.,   1.,   1.,   2.,  50.,  99.,  99.,
         99.,  99.,  50.,   1.,   1.,  50.,  99.,  99.,  99.,  99.,  99.,  99.,
         99.,  99.,  99.,  99.,  99.,  99.,  99.], grad_fn=<RoundBackward>)

In [ ]:
mu = torch.randn(k,  requires_grad=True)

In [609]:
optimizer = torch.optim.SGD([mu], lr=1e-2)
for t in range(0,10000):
    x = torch.cat((torch.tensor([1.0]), torch.sigmoid(mu)))
    all_equi = torch.prod(x) + torch.prod(1.0 - x)
    loss = -torch.log(all_equi)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
loss, torch.round(x*100)

(tensor(0.0604, grad_fn=<NegBackward>),
 tensor([100., 100., 100., 100., 100., 100., 100., 100., 100., 100., 100., 100.,
         100., 100., 100., 100., 100., 100., 100., 100., 100., 100., 100., 100.,
         100., 100., 100., 100., 100., 100., 100.], grad_fn=<RoundBackward>))

In [647]:
mu = torch.randn(k,  requires_grad=True)
x = torch.cat((torch.tensor([1.0]), torch.sigmoid(mu)))

def compute_all_equi(x):
    all_equi_true = 1.0
    all_equi_false = 1.0
    for i in range(0,k+1):
        prev = i-1 % k
        all_equi_dumb = all_equi * ((x[i] * x[prev]) + (1-x[i]) * (1-x[prev]))
        all_equi_true = all_equi_true * x[i]
        all_equi_false = all_equi_false * (1-x[i])
    return all_equi_false + all_equi_true
compute_all_equi(x)

tensor(5.6635e-10, grad_fn=<ThAddBackward>)

In [659]:
optimizer = torch.optim.SGD([mu], lr=1e-1)
for t in range(0,100):
    x = torch.cat((torch.tensor([1.0]), torch.sigmoid(mu)))
#     print(x)
    all_equi = compute_all_equi(x)
    loss = -torch.log(all_equi)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
#     print(loss)

loss, torch.round(x*100)

(tensor(0.2546, grad_fn=<NegBackward>),
 tensor([100.,  99.,  99.,  99.,  99.,  99.,  99.,  99.,  99.,  99.,  99.,  99.,
          99.,  99.,  99.,  99.,  99.,  99.,  99.,  99.,  99.,  99.,  99.,  99.,
          99.,  99.,  99.,  99.,  99.,  99.,  99.], grad_fn=<RoundBackward>))

In [157]:
all_equi(mu)

0 3.2000861167907715
1 3.196314573287964
2 3.1925437450408936
3 3.1887729167938232
4 3.1850016117095947
5 3.181230306625366
6 3.1774587631225586
7 3.173686981201172
8 3.169914484024048
9 3.166141986846924
10 3.1623682975769043
11 3.1585938930511475
12 3.1548187732696533
13 3.151042938232422
14 3.147265672683716
15 3.1434876918792725
16 3.1397082805633545
17 3.135927200317383
18 3.1321451663970947
19 3.128361225128174
20 3.1245763301849365
21 3.120788812637329
22 3.116999864578247
23 3.1132092475891113
24 3.1094162464141846
25 3.105621337890625
26 3.1018242835998535
27 3.09802508354187
28 3.0942232608795166
29 3.090419292449951
30 3.0866124629974365
31 3.082803249359131
32 3.078991413116455
33 3.075176477432251
34 3.0713589191436768
35 3.067538022994995
36 3.0637142658233643
37 3.059887409210205
38 3.0560572147369385
39 3.0522236824035645
40 3.048387050628662
41 3.044546604156494
42 3.0407028198242188
43 3.036855459213257
44 3.03300404548645
45 3.029149055480957
46 3.02528977394104
47 3

tensor(0.0602, grad_fn=<ProdBackward1>)

In [78]:
eq_12 = x[0] * x[1] + (1-x[0])*(1 - x[1])
eq_12_true = x[0] * x[1]
eq_12_false = (1-x[0]) * (1-x[1])
eq_123_true =  eq_12_true * x[2]
eq_123_false = eq_12_false * (1-x[2])
eq_23_true = x[1] * x[2]
eq_23_false = (1-x[1])*(1-x[2])
eq_123_true, eq_12_true * eq_23_true

(tensor(0.1128), tensor(0.0531))